# Computation on NumPy Arrays: Universal Functions

Data Science에서 NumPy와 같은 패키지를 사용하는 이유를 이해하는 것도 중요하다.
- 수 배열을 편리하게 사용할 수 있다.
- NumPy의 *universal functions* (ufuncs) 을 토대로 구현된 *vectorized* operation을 사용하면 빠른 계산이 가능하다. 

## The Slowness of Loops

CPython이라는 이름의 Python의 기본 구현은 매우 느린 편이다. 이는 동적 타입을 지원하는 인터프리터 방식으로 실행되기 때문이다. (참고: 컴파일러 vs 인터프리터)

최근 그러한 문제점을 해결하기 위한 프로젝트들이 진행중이지만 아직 대중적으로 사용되지는 않고 있다. 
-  [PyPy](http://pypy.org/) project (just-in-time compiled implementation of Python)
-  [Cython](http://cython.org) project (Python code를 compilable C code 변환)
-  [Numba](http://numba.pydata.org/) project (Python snippet을 LLVM bytecode로 변환)

느린 Pythhon 원인 중 특히 Loop 계산이 있다.

In [2]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values)) # output이라는 변수에 values의 길이만큼의 사이즈를 가지는 배열을 할당
    for i in range(len(values)): # i에 0부터 len(values)-1까지 반복
        output[i] = 1.0 / values[i] # output배열에 values라는 배열의 인덱스 대입
    return output # output 반환
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

위 코드는 논리적으로 문제가 없지만 매우 큰 배열에 대해 실행하면 매우 느려진다. 

(IPython의 ``%timeit`` line magic 명령을 통해 시간 체크가 가능하다.)

%timeit = 실행 시간 확인할 수 있는 magic function

In [2]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array) # 시간을 측정하는 line magic

KeyboardInterrupt: 

## Introducing UFuncs

NumPy의 Vectorized operation은 loop 계산과정을 compiled layer로 실행시킴으로써 속도가 빠르다.

In [ ]:
print(compute_reciprocals(values)) 
print(1.0 / values) # valuse는 array => Numpy에서 제공하는 Vectorized operation
# 각각의 원소를 대입해서 결과 도출하는 것임

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


다음 실행 시간을 비교해 보자.

In [ ]:
%timeit (1.0 / big_array)

2.4 ms ± 303 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


두 array사이의 vectorized operation도 가능하다. 

In [ ]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

Multi-dimensional array도 가능하다.

In [ ]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

Loop를 통한 배열 계산을 만나면 항상 vectorized expression을 생각하자.

## Exploring NumPy's UFuncs

아래에서 자세하게 살펴보자. 
- *unary ufuncs* : input이 하나인 경우
- *binary ufuncs* : input이 2개인 경우

### Array arithmetic

addition, subtraction, multiplication, and division.

In [ ]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # floor division

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]


In [ ]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


In [ ]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

사실 산술연산자는 단순히 해당 함수의 wrapper이다. (아래 표 참조)

예) ``+`` operator는  ``add`` function의 wrapper이다.

오히려 함수가 유용한 경우가 있다.

In [ ]:
np.add(x, 2)

array([2, 3, 4, 5])



| Operator	    | Equivalent ufunc    | Description                           |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |Addition (e.g., ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |Subtraction (e.g., ``3 - 2 = 1``)      |
|``-``          |``np.negative``      |Unary negation (e.g., ``-2``)          |
|``*``          |``np.multiply``      |Multiplication (e.g., ``2 * 3 = 6``)   |
|``/``          |``np.divide``        |Division (e.g., ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |Floor division (e.g., ``3 // 2 = 1``)  |
|``**``         |``np.power``         |Exponentiation (e.g., ``2 ** 3 = 8``)  |
|``%``          |``np.mod``           |Modulus/remainder (e.g., ``9 % 4 = 1``)|


### Absolute value



In [ ]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

array([2, 1, 0, 1, 2])

해당 NumPy ufunc 은  ``np.absolute``이며 alias ``np.abs``가 가능하다.

In [ ]:
np.absolute(x)

array([2, 1, 0, 1, 2])

In [ ]:
np.abs(x)

array([2, 1, 0, 1, 2])

복소수(complex data)인 경우 크기(magnitude)를 계산한다.

In [ ]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([5., 5., 2., 1.])

### Trigonometric functions

다음은 삼각함수에 대한 설명이다.

In [ ]:
theta = np.linspace(0, np.pi, 3)

In [ ]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


In [ ]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### Exponents and logarithms

다음은 지수 및 로그함수이다.

In [ ]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [ 2.71828183  7.3890561  20.08553692]
2^x   = [2. 4. 8.]
3^x   = [ 3  9 27]


In [ ]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


In [ ]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) = [0.         0.0009995  0.00995033 0.09531018]


위 함수는 ``np.log`` 혹은 ``np.exp``보다 x가 매우 작은 경우 더 정밀하게 계산해 준다.

## Advanced Ufunc Features


### Specifying output

큰 데이터 계산일 경우 출력 장소를 지정하면 임시 저장소를 만들지 않으므로 효율적이다.

ufuncs 내 ``out`` argument를 사용하여 지정한다.

In [ ]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y) #for를 사용하지않고 사용한 효과를 냄. 메모리 할당이 아닌 y로 바로 대입이기에 속도적 우위
print(y)

[ 0. 10. 20. 30. 40.]


In [ ]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


다시 말해, 위 코드와 달리 ``y[::2] = 2 ** x``라고 한다면 ``2 ** x`` 결과가 임시 array에 저장되고 다시 ``y`` array로 복사되므로 매우 큰 데이터일 경우 비효율적이다.


### Aggregates

Binary opearator인 경우 *reduce* method를 통해 array내 모든 element에 대해 반복하여 결과를 줄일 수 있다. 

In [ ]:
x = np.arange(1, 6)
np.add.reduce(x)

15

In [ ]:
np.multiply.reduce(x)

120

모든 중간 결과를 저장하는 경우에는 ``accumulate``를 사용하면 된다.

In [ ]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15])

In [ ]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120])

``np.sum``, ``np.prod``, ``np.cumsum``, ``np.cumprod``와 같은 함수도 마찬가지이며 다음 절에서 살펴본다. 

### Outer products

두 vector의 outer product 계산도 가능하다. (행렬의 외적)

In [8]:
x = np.arange(1, 6)
np.multiply.outer(x, x)
print(x)
print(np.add.accumulate(x))

[1 2 3 4 5]
[ 1  3  6 10 15]


AttributeError: module 'numpy' has no attribute 'accumulate'

``np.outer``, ``np.inner`` 함수도 사용이 가능하다.

In [6]:
np.inner(x, x) # 벡터의 내적은 곱의 합

55

In [7]:
np.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])